# Fingernail Classification

* 손톱 상태에 기반한 자가 건강 진단을 해볼 수 있는 웹 서비스의 프로토타입을 구축해보고자 한다. 
* DeepLearning의 CNN을 활용하였으며 수집한 데이터의 수가 적기 때문에 Transfer Learning 기법을 이용했다.

# Convolutional Neural Network(CNN) with Keras

### Transfer Learning - MobileNet

In [1]:
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import warnings
warnings.filterwarnings('ignore')

In [2]:
base_model = MobileNet(weights='imagenet', include_top=False)

In [3]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.7)(x)
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
preds=Dense(7, activation='softmax')(x)

In [4]:
model=Model(inputs=base_model.input, outputs=preds)

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, None, None, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, None, None, 32)    864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, None, None, 32)    128       
_________________________________________________________________
conv1_relu (ReLU)            (None, None, None, 32)    0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, None, None, 32)    288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, None, None, 32)    128       
__________

In [6]:
for layer in model.layers:
    layer.trainable=True

In [7]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory('/Users/LG/Desktop/finger',
                                                   target_size=(224, 224),
                                                   color_mode='rgb',
                                                   batch_size=32,
                                                   class_mode='categorical',
                                                   shuffle=True)

Found 554 images belonging to 7 classes.


In [8]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

step_size_train=train_generator.n//train_generator.batch_size
model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   epochs=10,
                   validation_data=train_generator)

Epoch 1/10
17/17 [==============================] - 260s 15s/step - loss: 1.9472 - acc: 0.3193 - val_loss: 2.1326 - val_acc: 0.4061
Epoch 2/10
17/17 [==============================] - 234s 14s/step - loss: 1.3307 - acc: 0.5296 - val_loss: 1.5676 - val_acc: 0.5217
Epoch 3/10
17/17 [==============================] - 234s 14s/step - loss: 0.9431 - acc: 0.6585 - val_loss: 2.2006 - val_acc: 0.4188
Epoch 4/10
17/17 [==============================] - 230s 14s/step - loss: 0.6517 - acc: 0.7751 - val_loss: 1.4690 - val_acc: 0.5578
Epoch 5/10
17/17 [==============================] - 234s 14s/step - loss: 0.5379 - acc: 0.8382 - val_loss: 1.0999 - val_acc: 0.6625
Epoch 6/10
17/17 [==============================] - 226s 13s/step - loss: 0.3150 - acc: 0.9056 - val_loss: 0.6674 - val_acc: 0.8032
Epoch 7/10
17/17 [==============================] - 221s 13s/step - loss: 0.2892 - acc: 0.8942 - val_loss: 0.8747 - val_acc: 0.7509
Epoch 8/10
17/17 [==============================] - 229s 13s/step - loss: 0.

### 실험내용

* 데이터의 수가 부족할 때 Transfer Learning을 이용하면 좋은 결과가 나온다고 알려져 있다. Finger Classification을 할 때 Transfer Learning을 이용하였고 Layer의 구성을 바꾸어 가며 Test Error를 확인해 보았다.

* CNN의 Fully Connected Layer를 자유롭게 구성하였다. Convolution Layer와 Pooling Layer는 MobileNet의 Layer구성을 그대로 이용하여 Weight값은 이미 학습되어 있다. 학습시키는 대상은 Fully Connected Layer뿐이다.

* Fully Connected Layer에 Dense(1024, activation='relu') 두 개, Dense(512, activation='relu')한 개, Softmax를 두었을 때 train accuracy는 94%를 보였지만 test accuracy는 40%에 불과했다. 이는 Overfitting된 것으로 보고, drop out layer를 추가하여 재구성 하였다.

* 똑같은 train_generator를 validation에 사용하였다 하더라도 객체 내부에서 랜덤하게 augmentation을 진행하여 다른 dataset을 구성해주기 때문에 testset으로 보아도 지장이 없다.

* Fully Connected Layer의 개수를 늘리는 것은 좋은 방안이 아닌 듯 했다. Drop out rate를 0.3, 0.5, 0.7으로 각각 적용해 보았을 때, 0.5가 비교적 적당한 것 같았다. 데이터의 수가 적어서 큰 rate는 적절하지 않은 듯 했다. FCL의 개수를 늘리지 않고 DOR=0.5의 layer를 하나 추가했을 때 test accuracy가 최고 60%, 평균 50%을 넘는 것을 확인할 수 있었다. Fully Connected Layer의 넓이(input의 개수)가 크면 클 수록 weight값들의 다양성이 늘어나 overfitting에 영향을 주지않을까 생각되어 줄여보았으나 별 차이가 없었다.

* 오히려 전체를 학습시켰을 때 결과가 많이 달라졌고 80% 이상의 test accuracy를 보인 것을 확인 했다. test accuracy가 80%이상의 수치를 보였지만 여전히 변동이 심했다.

* 데이터의 수가 적으면서 분류되는 class의 개수가 비교적 많은 것이 overfitting이 계속되는 가장 큰 이유이다. 좀더 완벽한 모델구성을 위해서는 데이터의 개수를 늘리는 것이 급선무 이다.

* 우선 결과가 가장 좋은 모델을 저장시켜둔 후에 웹에서 사용할 수 있도록 한다.

* __참고__ http://incredible.ai/artificial-intelligence/2017/05/13/Transfer-Learning
* __참고__ https://towardsdatascience.com/transfer-learning-from-pre-trained-models-f2393f124751
* __참고__ https://datascience.stackexchange.com/questions/47966/over-fitting-in-transfer-learning-with-small-datase

### 결과확인

In [30]:
CNN_finger_model = model.to_json()
with open("CNN_finger_model", "w") as json_file:
    json_file.write(CNN_finger_model)
model.save_weights("CNN_finger_weight.h5")

In [31]:
from tensorflow.keras.models import model_from_json

reader = open('CNN_finger_model', 'r')
CNN_finger_model = reader.read()
reader.close()

CNN_finger_model = model_from_json(CNN_finger_model)
CNN_finger_model.load_weights("CNN_finger_weight.h5")

In [32]:
import numpy as np
img = image.load_img('/Users/LG/Desktop/finger/0/7.jpg', target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
pred = CNN_finger_model.predict(x).argmax()
pred

0